# implementing the graph data-structure, 2-OPT algorithm : 

In [1]:
# Adjacency Matrix representation in Python

class Graph(object):

   
    def __init__(self, size):
        """initialise the graph object using an adjacency matrix
           _____________________________________________________
           
           parameters : 
           
           size = the dimension of the matrix to be created. 
            
        """
        self.adjMatrix = []
        for i in range(size):
            self.adjMatrix.append([0 for i in range(size)])
        self.size = size

    
    def add_edge(self, v1, v2):
        """adding an edge in the graph between vertices v1, v2.
           _____________________________________________________
           
           parameters : 
           
           v1 = the first vertex
           v2 = the second vertex
            
        """
        if v1 == v2:
            print("Same vertex %d and %d" % (v1, v2))
        
        self.adjMatrix[v1][v2] = 1
        self.adjMatrix[v2][v1] = 1

    
    def remove_edge(self, v1, v2):
        """if there is an edge b/w v1, v2 then remove the edge 
           between the vertices v1, v2.
           ____________________________________________________
           
           parameters : 
           
           v1, v2 : indices of the vertices from which edge has 
                    to be removed. 
        """
        
        if self.adjMatrix[v1][v2] == 0:
            print("No edge between %d and %d" % (v1, v2))
            return
        self.adjMatrix[v1][v2] = 0
        self.adjMatrix[v2][v1] = 0

    def __len__(self):
        # returns the size of the graph 
        
        return self.size

    
    def print_matrix(self):
        # prints the adjacency matrix . 
        
        for row in self.adjMatrix:
            for val in row:
                print('{:4}'.format(val)),
            print

we could have used this data-structure, but we have better ways of working with the python libraries such as pandas and numpy, so we will abstain from using the above implementation, but if we had to then the only change we have to do is to populate the matrix with the cost of travel between the cities connected by an edge instead of the binary variable 0, 1 which just denotes the presence of an edge b/w two vertices

In [2]:
import numpy as np
import pandas as pd

In [3]:
import import_ipynb

In [5]:
from extraction_preprocessing_visualisation import normalise_two, read_raw
from neuron import SELF_ORGANISING_MAP, route_find, generate_random_network_weights, get_gaussian_neighborhood, choose_closest, euclidean_distance

In [15]:
def dist(a, b):
        """Return the euclidean distance between cities tour[a] and tour[b]."""
        return np.hypot(float(coords[tour[a]][0]) - float(coords[tour[b]][0]), float(coords[tour[a]][1]) - float(coords[tour[b]][1]))

def two_opt_one():
    min_change = 0
    num_cities = len(tour)
    # Find the best move
    for i in range(num_cities - 2):
        for j in range(i + 2, num_cities - 1):
            change = dist(i, j) + dist(i+1, j+1) - dist(i, i+1) - dist(j, j+1)
            if change < min_change:
                min_change = change
                min_i, min_j = i, j
    # Update tour with best move
    if min_change < 0:
        tour[min_i+1:min_j+1] = tour[min_i+1:min_j+1][::-1]        
    return tour 

# testing this implementation on qatar dataset : 

In [16]:
cities = read_raw('/Users/adityagarg/Desktop/project.nosync/data/qa194.tsp')
coords = normalise_two(cities)[['x', 'y']].values
# dummy tour: 0, 1, 2, 3...
tour = np.array([i for i in range(len(coords))])
print("There are", len(coords), "cities in coords")

 the problem has 194 number of cities.
There are 194 cities in coords


In [18]:
%time new_tour = two_opt_one()

CPU times: user 226 ms, sys: 4.35 ms, total: 230 ms
Wall time: 230 ms


In [21]:
def convert_from_index_to_array(route):
    
    l = []
    for i in route: 
        l.append(int(i))
    return np.array(l)
    

def compare_routes(route_1, route_2):
    """ compares if the routes after the som and the route after the 2-opt are same or not 
    
        __________________________________________________________________________________
        
        prameters :
        
        1) route_1 = route given out by the SOM converted into an np.array
        2) route_2 = route generated after the 2-OPT algorithm applied to the previous route
    
    
    """
    return route_1 == route_2

# using cython :

In [27]:
%load_ext Cython

In [35]:
%%cython
import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport sqrt

cpdef two_opt_cython(double[:,:] coords, int[:] tour_):
    cdef float min_change, change
    cdef int i, j, min_i, min_j, num_cities
    num_cities = len(tour_)
    min_change = 0
    # Find the best move
    for i in range(num_cities - 2):
        for j in range(i + 2, num_cities - 1):
            change = dist(i, j, tour_, coords) + dist(i+1, j+1, tour_, coords)
            change = - dist(i, i+1, tour_, coords) - dist(j, j+1, tour_, coords)
            if change < min_change:
                min_change = change
                min_i, min_j = i, j
    # Update tour with best move
    if min_change < 0:
        tour_[min_i+1:min_j+1] = tour_[min_i+1:min_j+1][::-1]
    return np.asarray(tour_)  # memoryview to numpy array

cdef float dist(int a, int b, int[:] tour_view, double[:,:] coords_view):
    """Return the euclidean distance between cities tour[a] and tour[b]."""
    return sqrt((coords_view[tour_view[a], 0] - coords_view[tour_view[b], 0])**2 +
                (coords_view[tour_view[a], 1] - coords_view[tour_view[b], 1])**2)

In [36]:
%time two_opt_cython(coords, tour.astype('int32'))

CPU times: user 4.2 ms, sys: 876 µs, total: 5.08 ms
Wall time: 4.66 ms


array([ 22,  12,  15,   7,   5,   0,   2,   4,   8,   9,  11,  14,  18,
        29,  31,  30,  34,  41,  49,  54,  48,  43,  53,  52,  51,  47,
        45,  37,  40,  55,  57,  42,  39,  46,  33,  38,  26,  36,  50,
        60,  66,  72,  65,  67,  83,  78,  80,  76,  69,  63,  56,  44,
        28,  21,  27,  59,  32,  17,  20,  23,  25,  68,  73,  71,  77,
        74,  75,  70,  79,  86, 101, 102,  90,  92,  87,  82,  91,  96,
        94,  95, 105, 117, 104, 106, 107,  99, 109, 111, 114, 115, 116,
       120, 119, 122, 123, 127, 132, 128, 134, 142, 147, 135, 130, 150,
       154, 159, 165, 170, 184, 169, 179, 177, 167, 164, 166, 161, 157,
       158, 146, 151, 140, 121, 137, 138, 153, 143, 149, 152, 156, 176,
       180, 192, 187, 190, 191, 188, 183, 174, 172, 173, 178, 182, 186,
       189, 185, 193, 171, 181, 175, 168, 163, 162, 160, 148, 145, 141,
       136, 139, 144, 155, 129, 126, 131, 133, 124, 125, 113, 118, 112,
       108,  81, 100, 103, 110,  98,  93,  89,  88,  61,  58,  3